In [12]:
!pip3 install spacy scikit-learn pandas
import spacy.cli
spacy.cli.download("en_core_web_sm")

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [2]:
# mount to folder
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/mobile_privacy/cleaned/Kappa

Mounted at /content/drive
/content/drive/MyDrive/mobile_privacy/cleaned/Kappa


In [13]:
from lib_analysis import read_and_parse_data, is_consistent
from lib_analysis import read_data
from sklearn.metrics import cohen_kappa_score
import csv
import pandas as pd

In [16]:
def categorize(data, category):
    res = {}
    offset = -1
    bound = len(category)
    for k,v in data.items():
        new_code = list()
        for c in v['codes']:
            if c.startswith('o'):
                new_code.append(c)
                continue
            if c.startswith('b'):
                offset += 1
            new_code.append(c + str(category[offset]))
        v['codes'] = new_code
        res[k] = v
    return res
            
def cohen_kappa_categorized(filepath1, filepath2, category1=list(), category2=list()):
    data1 = read_and_parse_data(filepath1)
    data2 = read_and_parse_data(filepath2)
    if len(category1) > 0:
        data1 = categorize(data1, category1)
    if len(category2) > 0:
        data2 = categorize(data2, category2)

    scenario_ids = list(data1.keys())
    all_codes1 = [c for d in data1.values() for c in d['codes']]
    all_codes2 = [c for d in data2.values() for c in d['codes']]

    kappa = cohen_kappa_score(all_codes1, all_codes2)
    print('Cohen\'s Kappa, All Codes: %0.4f' % kappa)


In [44]:
# uncategorized
samples_name_r1 = ['sample1', 'sample2', 'sample3']
samples_name_r2 = ['sample1 re-annotated', 'sample2 re-annotated']
filepaths1_r1 = ['./samples/sample1-rater1.txt', './samples/sample2-rater1.txt', './samples/sample3-rater1.txt']
filepaths1_r2 = ['./samples/sample1-rater1-reann.json', './samples/sample2-rater1-reann.txt']
filepaths2_r1 = ['./samples/sample1-rater2.txt','./samples/sample2-rater2.txt', './samples/sample3-rater2.txt']
filepaths2_r2 = ['./samples/sample1-rater2-reann.json', './samples/sample2-rater2-reann.txt']

print("Round 1:\n")
for idx, filepath in enumerate(zip(filepaths1_r1, filepaths2_r1)):
    filepath1, filepath2 = filepath
    print(samples_name_r1[idx])
    cohen_kappa_categorized(filepath1, filepath2)
    print('\n')

print("\nRound 2:\n")
for idx, filepath in enumerate(zip(filepaths1_r2, filepaths2_r2)):
    filepath1, filepath2 = filepath
    print(samples_name_r1[idx])
    cohen_kappa_categorized(filepath1, filepath2)
    print('\n')


Round 1:

sample1
Cohen's Kappa, All Codes: 0.2971


sample2
Cohen's Kappa, All Codes: 0.5746


sample3
Cohen's Kappa, All Codes: 0.7745



Round 2:

sample1
Cohen's Kappa, All Codes: 0.7147


sample2
Cohen's Kappa, All Codes: 0.6941




In [45]:
# categorized
category1 = pd.read_excel('./samples/sample1-rater1-reann.xlsx')['tag'].to_list()
category2 = pd.read_excel('./samples/sample1-rater2-reann.xlsx')['tag'].to_list()
cohen_kappa_categorized('./samples/sample1-rater1-reann.json','./samples/sample1-rater2-reann.json', category1, category2)

Cohen's Kappa, All Codes: 0.6808
